In [1]:
from ib_insync import IB, Stock, MarketOrder, LimitOrder

ib = IB()
await ib.connectAsync("127.0.0.1", 7497, clientId=2)  # 7497 = paper TWS port


<IB connected to 127.0.0.1:7497 clientId=2>

Error 10089, reqId 18: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.AAPL NASDAQ.NMS/TOP/ALL, contract: Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')
Error 200, reqId 19: No security definition has been found for the request, contract: Stock(symbol='NOTAREALSYMBOL', exchange='SMART', currency='USD')


[IB ERROR] code=200 reqId=19 sym=NOTAREALSYMBOL msg=No security definition has been found for the request


In [2]:
ib.isConnected()

True

In [3]:
ib.managedAccounts()

['DUO986453']

In [4]:
paper_accounts = set(ib.managedAccounts())
print("Accounts:", paper_accounts)

Accounts: {'DUO986453'}


In [5]:
def assert_paper_account(account: str):
    assert account in paper_accounts, f"Account {account} not in managed accounts!"

In [7]:
import asyncio
from ib_insync import Stock, MarketOrder

contract = Stock("AAPL", "SMART", "USD")

# notebook-safe qualify:
qualified = await ib.qualifyContractsAsync(contract)
print("Qualified:", qualified[0])

acct = next(iter(paper_accounts))
assert_paper_account(acct)

order = MarketOrder("BUY", 1)
order.account = acct

trade = ib.placeOrder(contract, order)
print("Submitted:", trade)

# give it a moment to update
await asyncio.sleep(2)
print("Status:", trade.orderStatus.status, "filled:", trade.orderStatus.filled)

Qualified: Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')
Submitted: Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=12, clientId=2, action='BUY', totalQuantity=1, account='DUO986453'), orderStatus=OrderStatus(orderId=12, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2025, 12, 13, 4, 8, 46, 307190, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')
Status: PendingSubmit filled: 0.0


In [12]:
# Pull historical bars
from ib_insync import Stock, util

contract = Stock("AAPL", "SMART", "USD")
await ib.qualifyContractsAsync(contract)

bars = await ib.reqHistoricalDataAsync(
    contract,
    endDateTime="",
    durationStr="2 D",
    barSizeSetting="5 mins",
    whatToShow="TRADES",
    useRTH=True,
    formatDate=1
)

df = util.df(bars)
df.tail()

,date,open,high,low,close,volume,average,barCount
151,2025-12-12 15:35:00-05:00,278.03,278.05,276.94,277.24,465034.0,277.421,2473
152,2025-12-12 15:40:00-05:00,277.25,277.72,277.25,277.67,342011.0,277.546,1812
153,2025-12-12 15:45:00-05:00,277.68,278.21,277.62,277.90,442165.0,277.901,2908
154,2025-12-12 15:50:00-05:00,277.92,278.50,277.13,277.86,510544.0,277.788,3526
155,2025-12-12 15:55:00-05:00,277.87,278.38,277.77,278.37,900593.0,278.226,6060


In [13]:
from ib_insync import Stock
import asyncio

contract = Stock("AAPL", "SMART", "USD")
await ib.qualifyContractsAsync(contract)

t = ib.reqMktData(contract)
await asyncio.sleep(2)

# Many may be None after hours / without subs, but you learn the shape
(t.bid, t.ask, t.last, t.close, t.volume)

(nan, nan, nan, nan, nan)

In [ ]:
# Define and qualify a contract

In [ ]:
ib.disconnect()

In [19]:
import asyncio
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import pandas_market_calendars as mcal
from ib_insync import IB, Stock, util

TZ_LOCAL = ZoneInfo("America/Denver")
TZ_MKT = ZoneInfo("America/New_York")

SYMBOLS = ["AAPL", "MSFT", "AMZN"]  # your 3 companies

# --- connect (paper) ---
ib = IB()
await ib.connectAsync("127.0.0.1", 7497, clientId=41, timeout=10)

# --- trading sessions (NYSE calendar) ---
cal = mcal.get_calendar("XNYS")
today = datetime.now(TZ_MKT).date()
sched = cal.schedule(start_date=today - timedelta(days=14), end_date=today)
sessions = sched.tail(5)  # last 5 sessions (typically Mon–Fri)

sessions

,market_open,market_close
2025-12-08,2025-12-08 14:30:00+00:00,2025-12-08 21:00:00+00:00
2025-12-09,2025-12-09 14:30:00+00:00,2025-12-09 21:00:00+00:00
2025-12-10,2025-12-10 14:30:00+00:00,2025-12-10 21:00:00+00:00
2025-12-11,2025-12-11 14:30:00+00:00,2025-12-11 21:00:00+00:00
2025-12-12,2025-12-12 14:30:00+00:00,2025-12-12 21:00:00+00:00


In [20]:
def ensure_tz(dt_series):
    # bars often come back as naive datetimes; assume local machine tz
    s = pd.to_datetime(dt_series)
    if s.dt.tz is None:
        s = s.dt.tz_localize(TZ_LOCAL)
    return s.dt.tz_convert(TZ_MKT)

def price_at_or_before(df_idx, t, col):
    # exact minute preferred, otherwise nearest prior minute
    if t in df_idx.index:
        return float(df_idx.loc[t, col])
    prior = df_idx[df_idx.index <= t]
    if prior.empty:
        return None
    return float(prior.iloc[-1][col])

async def bars_1m_last_week(symbol):
    c = Stock(symbol, "SMART", "USD")
    await ib.qualifyContractsAsync(c)
    bars = await ib.reqHistoricalDataAsync(
        c,
        endDateTime="",
        durationStr="7 D",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=True,      # regular trading hours only
        formatDate=1
    )
    df = util.df(bars)
    df["dt_mkt"] = ensure_tz(df["date"])
    df = df.set_index("dt_mkt").sort_index()
    return df

# fetch bars for all 3
bars_by_sym = {sym: await bars_1m_last_week(sym) for sym in SYMBOLS}

rows = []
for day, r in sessions.iterrows():
    open_mkt = r["market_open"].tz_convert(TZ_MKT).replace(second=0, microsecond=0)
    close_mkt = r["market_close"].tz_convert(TZ_MKT).replace(second=0, microsecond=0)
    midday_mkt = (open_mkt + (close_mkt - open_mkt) / 2).replace(second=0, microsecond=0)

    # for "close", use the last 1-min bar that ENDS at close -> starts at close - 1 minute
    close_bar_time = close_mkt - timedelta(minutes=1)

    for label, t, col in [
        ("open", open_mkt, "open"),
        ("midday", midday_mkt, "close"),
        ("close", close_bar_time, "close"),
    ]:
        row = {"date": open_mkt.date().isoformat(), "event": label}
        for sym in SYMBOLS:
            row[sym] = price_at_or_before(bars_by_sym[sym], t, col)
        rows.append(row)

table = pd.DataFrame(rows).sort_values(["date", "event"])
table

,date,event,AAPL,MSFT,AMZN
2,2025-12-08,close,277.96,491.12,226.99
1,2025-12-08,midday,277.22,490.66,227.26
0,2025-12-08,open,278.11,484.89,229.64
5,2025-12-09,close,277.26,492.06,227.90
4,2025-12-09,midday,278.00,490.08,227.71
3,2025-12-09,open,278.16,489.10,226.84
8,2025-12-10,close,278.72,478.52,231.69
7,2025-12-10,midday,278.46,481.80,230.90
6,2025-12-10,open,277.84,483.97,228.81
11,2025-12-11,close,278.06,483.69,230.27


In [21]:
table.to_csv("open_mid_close_last_week.csv", index=False)

In [22]:
import pandas as pd

# Load (or if df already exists, skip)
df = pd.read_csv("open_mid_close_last_week.csv")

symbols = ["AAPL", "MSFT", "AMZN"]

# Pivot to get open/close columns per date
wide = df.pivot(index="date", columns="event", values=symbols)

# Compute open->close returns per symbol
out = pd.DataFrame(index=wide.index)
for sym in symbols:
    o = wide[(sym, "open")]
    c = wide[(sym, "close")]
    out[f"{sym}_ret"] = (c - o) / o

# Day score = average (or sum) of the three returns
out["day_score"] = out[[f"{s}_ret" for s in symbols]].mean(axis=1)

# Rank best to worst
ranked = out.sort_values("day_score", ascending=False).reset_index()

# Pretty formatting (optional)
ranked_pct = ranked.copy()
for sym in symbols:
    ranked_pct[f"{sym}_ret"] = (ranked_pct[f"{sym}_ret"] * 100).round(3)
ranked_pct["day_score"] = (ranked_pct["day_score"] * 100).round(3)

print(ranked_pct)
print("\nBest day:", ranked_pct.loc[0, "date"], "score(%):", ranked_pct.loc[0, "day_score"])

         date  AAPL_ret  MSFT_ret  AMZN_ret  day_score
0  2025-12-11    -0.373     1.481    -0.290      0.273
1  2025-12-09    -0.324     0.605     0.467      0.250
2  2025-12-10     0.317    -1.126     1.259      0.150
3  2025-12-08    -0.054     1.285    -1.154      0.026
4  2025-12-12     0.166    -0.306    -1.601     -0.581

Best day: 2025-12-11 score(%): 0.273
